In [2]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import KFold

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from utils.utils import plot, set_all_seeds
from utils.kfold import KFoldDataModule, KFoldTrainer

In [4]:
checkpoint = 'roberta-base'
task = ['empathy', 'wrong_empathy']
# feature_to_tokenise=['demographic_essay', 'article']
# feature_to_tokenise=['demographic', 'essay']
feature_to_tokenise=['demographic_essay']
seed = 0
anno_diff_range = np.arange(0, 6.5, 0.5)

#################### v1 dataset ###################
filename = './data/v1-90-percent.tsv'

In [5]:
kfold_results = pd.DataFrame()

set_all_seeds(seed)

data_module = KFoldDataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=feature_to_tokenise,
    seed=seed
)

data = data_module.get_data(file=filename)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

for fold, (train_idx, test_idx) in enumerate(kfold.split(data)):
    print('Fold:', fold)
    
    train_loader = data_module.kfold_dataloader(
        file=filename, idx=train_idx
    )
    dev_loader = data_module.kfold_dataloader(
        file=filename, idx=test_idx
    )

    for anno_diff in anno_diff_range:
        trainer = KFoldTrainer(
            task=task,
            checkpoint=checkpoint,
            lr=1e-5,
            n_epochs=10,
            train_loader=train_loader,
            dev_loader=dev_loader,
            dev_label_gpt=filename,
            dev_label_crowd=None,
            device_id=0,
            anno_diff=anno_diff,
            mode=0 # -1: crowd, 1: gpt, 0: crowd-gpt
        )
    
        val_pearson_r = trainer.fit(dev_alpha=False)
    
        # save as seed in index and anno_diff in columns
        print(f'\n----Pearson r: {val_pearson_r}----\n')
        kfold_results.loc[fold, anno_diff] = val_pearson_r

    # # Saving in each fold to be cautious
    kfold_results.to_csv('./v1-kfold_results_anno_diff.tsv', sep='\t')

Map:   0%|          | 0/1670 [00:00<?, ? examples/s]

Fold: 0


Map:   0%|          | 0/1670 [00:00<?, ? examples/s]

Map:   0%|          | 0/1670 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch: 1
Train loss: 12.469064418305742
Pearson r: 0.872
Validation loss: 0.36246755719184875
Best Pearson r: 0.872

Epoch: 2
Train loss: 3.450895530112246
Pearson r: 0.874
Validation loss: 0.37363144755363464
Best Pearson r: 0.874

Epoch: 3
Train loss: 3.3167530450415104
Pearson r: 0.874
Validation loss: 0.3706496059894562
Best Pearson r: 0.874

Epoch: 4
Train loss: 3.315901022008125
Pearson r: 0.874
Validation loss: 0.3987455666065216

----Pearson r: 0.874----



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Train loss: 11.588499340605228
Pearson r: 0.874
Validation loss: 0.3484444320201874
Best Pearson r: 0.874

Epoch: 2
Train loss: 3.4113551428977478
Pearson r: 0.874
Validation loss: 0.3643069565296173
Best Pearson r: 0.874

Epoch: 3
Train loss: 3.250780796751063
Pearson r: 0.893
Validation loss: 0.29399076104164124
Best Pearson r: 0.893

Epoch: 4
Train loss: 2.77553114358415
Pearson r: 0.894
Validation loss: 0.3357246220111847
Best Pearson r: 0.894

Epoch: 5
Train loss: 2.419114201626879
Pearson r: 0.899
Validation loss: 0.299678772687912
Best Pearson r: 0.899

Epoch: 6
Train loss: 2.1241658114372415
Pearson r: 0.896
Validation loss: 0.2834005653858185
Best Pearson r: 0.899

Epoch: 7
Train loss: 1.8762602806091309
Pearson r: 0.894
Validation loss: 0.30081892013549805
Best Pearson r: 0.899

Epoch: 8
Train loss: 1.6131584701385904
Pearson r: 0.891
Validation loss: 0.33059629797935486
Best Pearson r: 0.899

Epoch: 9
Train loss: 1.4682482205172802
Pearson r: 0.885
Validation loss: 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Train loss: 11.769449807227925
Pearson r: 0.873
Validation loss: 0.36714956164360046
Best Pearson r: 0.873

Epoch: 2
Train loss: 3.541997215849288
Pearson r: -0.017
Validation loss: 216.5758056640625
Best Pearson r: 0.873

Epoch: 3
Train loss: 3.392550268071763
Pearson r: 0.874
Validation loss: 0.36107248067855835
Best Pearson r: 0.874

Epoch: 4
Train loss: 3.1182480289580976
Pearson r: 0.895
Validation loss: 0.29299312829971313
Best Pearson r: 0.895

Epoch: 5
Train loss: 2.6080402275349233
Pearson r: 0.896
Validation loss: 0.29282188415527344
Best Pearson r: 0.896

Epoch: 6
Train loss: 2.2756569366505803
Pearson r: 0.889
Validation loss: 0.30990391969680786
Best Pearson r: 0.896

Epoch: 7
Train loss: 2.0184028833470444
Pearson r: 0.885
Validation loss: 0.3181782066822052

----Pearson r: 0.896----

